Eta korar por must kernel restart korte hobe

In [1]:
!pip install bitsandbytes accelerate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/54.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/54.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/54.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/54.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/54.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/54.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/54.7 MB 409.0 kB/s eta 0:02:13
   ---------------------------------------- 0.5/54.7 MB 409.0 kB/s eta 0:02:13
   ---------------------------------------- 0.5/54.7 MB 409.0 kB/s eta 0:02:13
   ---------------------------------------- 0.5/54.7 MB 409.0 kB/s eta 0:02:13
   ---------------------------------------- 0.5/54.7 MB 409.0 kB/s eta 0:02:13
    --------------------------------------- 0.8/54.7 MB 316.4 kB/s eta 0:02:51
    -----------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 📊 ধাপ ১: মেমোরি স্ট্যাটাস
কোয়ান্টাইজেশনের আগে দেখে নিচ্ছি আমাদের জিপিউতে কতটুকু জায়গা খালি আছে।

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# VRAM স্ট্যাটাস চেক
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    free_mem = torch.cuda.mem_get_info()[0] / 1024**3
    print(f"Free VRAM before loading: {free_mem:.2f} GB")

CUDA Available: True
GPU: NVIDIA RTX A4500
Free VRAM before loading: 18.52 GB


### 📉 ধাপ ২: ৪-বিট মোডে মডেল লোড (Quantization)
এখানে আমরা `bitsandbytes` লাইব্রেরি ব্যবহার করে মডেলটিকে সংকুচিত (Compress) করছি।
- **`load_in_4bit=True`**: মডেলের প্রতিটি প্যারামিটারকে ১৬ বিটের বদলে ৪ বিটে নামিয়ে আনে।
- **`bnb_4bit_quant_type="nf4"`**: 'Normalized Float 4' - এটি বিশেষ একটি ডেটা টাইপ যা তথ্যের ক্ষতি না করে সাইজ কমায়।
- **`bnb_4bit_use_double_quant=True`**: এটি মেমোরি আরও সাশ্রয় করে।
**ফলাফল:** মডেলের সাইজ ১৪ জিবি থেকে কমে **৫.১৮ জিবিতে** নেমে আসবে!

In [2]:
# মডেল নাম
model_name = "Qwen/Qwen2.5-7B-Instruct"

# ৪-বিট কোয়ান্টাইজেশন কনফিগারেশন (ফাইল থেকে নেওয়া)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, # তোমার Ampere GPU এর জন্য সেরা
    bnb_4bit_use_double_quant=True,       # মেমোরি আরও বাঁচানোর জন্য ডাবল কোয়ান্টাইজেশন
    bnb_4bit_quant_type="nf4"             # নরমালাইজড ফ্লোট ৪
)

print("Loading model in 4-bit mode... (This saves huge VRAM)")

# মডেল লোড
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)

# টোকেনাইজার লোড
tokenizer = AutoTokenizer.from_pretrained(model_name)
# ব্যাচ প্রসেসিংয়ের জন্য প্যাডিং টোকেন সেট করা জরুরি
tokenizer.pad_token = tokenizer.eos_token 

print("Model Loaded Successfully!")

# লোড করার পর মেমোরি চেক
allocated = torch.cuda.memory_allocated(0) / 1024**3
print(f"VRAM Usage (4-bit): {allocated:.2f} GB") 
# ১৪ জিবির বদলে এখন মাত্র ~৫.৫ জিবি নেওয়ার কথা!

Loading model in 4-bit mode... (This saves huge VRAM)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model Loaded Successfully!
VRAM Usage (4-bit): 5.18 GB


### ⚡ ধাপ ৩: ব্যাচ জেনারেশন (Batch Processing)
এখানে আমরা লুপ (Loop) ব্যবহার না করে একসাথে অনেকগুলো প্রশ্ন প্রসেস করব।
- **`padding=True`**: ব্যাচের সব প্রশ্নের দৈর্ঘ্য সমান না হলে ছোট প্রশ্নগুলোর শেষে খালি টোকেন (pad token) যোগ করে সমান করা হয়, যাতে ম্যাট্রিক্স ক্যালকুলেশন ঠিক থাকে।
- **প্যারালাল প্রসেসিং:** GPU এর হাজার হাজার কোর ব্যবহার করে সে একই সময়ে ৫-১০টি প্রশ্নের উত্তর প্রসেস করে। এটি লুপের চেয়ে অনেক গুণ ফাস্ট।

In [3]:
def batch_generate(prompts, max_new_tokens=128):
    print(f"Processing {len(prompts)} prompts in a single batch...")
    
    # ১. চ্যাট টেমপ্লেট অ্যাপ্লাই করা (প্রতিটি প্রম্পটের জন্য)
    formatted_prompts = [
        tokenizer.apply_chat_template(
            [{"role": "user", "content": p}], 
            tokenize=False, 
            add_generation_prompt=True
        )
        for p in prompts
    ]

    # ২. টোকেনাইজ এবং প্যাডিং (ব্যাচ ইনপুট তৈরির জন্য)
    # padding=True মানে সব ইনপুট সমান সাইজের হবে
    inputs = tokenizer(
        formatted_prompts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True
    ).to(model.device)

    # ৩. জেনারেশন (একই সাথে সব উত্তর তৈরি হবে)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id # ব্যাচ প্রসেসিংয়ে এটি জরুরি
        )

    # ৪. ডিকোড করা (ইনপুট অংশ বাদ দিয়ে)
    # generated_ids থেকে ইনপুট অংশ স্লাইস করে ফেলা হচ্ছে
    input_length = inputs.input_ids.shape[1]
    output_ids = generated_ids[:, input_length:]
    
    responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    
    return responses

### ⏱️ ধাপ ৪: লাইভ টেস্ট
এখানে আমরা ৫টি প্রশ্ন একসাথে পাঠিয়ে দেখব মোট কত সময় লাগে।
- এটি প্রমাণ করবে যে ৪-বিট কোয়ান্টাইজেশন এবং ব্যাচিং ব্যবহার করলে RTX A4500 এর মতো কার্ডেও প্রোডাকশন-লেভেল স্পিড পাওয়া সম্ভব।

In [4]:
# প্রশ্নের তালিকা
my_questions = [
    "What is the capital of Bangladesh?",
    "Explain Python in one sentence.",
    "Who wrote Hamlet?",
    "What is 2 + 2?",
    "Name a primary color."
]

# ব্যাচ ফাংশন কল
import time
start_time = time.time()

answers = batch_generate(my_questions)

end_time = time.time()

# ফলাফল প্রিন্ট
print("-" * 50)
for q, a in zip(my_questions, answers):
    print(f"Q: {q}")
    print(f"A: {a}")
    print("-" * 30)

print(f"Total Time Taken: {end_time - start_time:.2f} seconds")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing 5 prompts in a single batch...
--------------------------------------------------
Q: What is the capital of Bangladesh?
A: I'd be happy to tell you that the capital of Bangladesh is Dhaka.
------------------------------
Q: Explain Python in one sentence.
A: !!");
Python is a high-level, interpreted programming language known for its readability and versatility, widely used for web development, data analysis, artificial intelligence, and more.
------------------------------
Q: Who wrote Hamlet?
A: To answer your question, "Hamlet" was written by William Shakespeare, one of the most famous and influential English-language playwrights in history. Shakespeare wrote "Hamlet" around 1600-1601, and it is one of his most renowned plays, known for its complex characters, themes, and psychological depth.
------------------------------
Q: What is 2 + 2?
A: 2 + 2 equals 4.
------------------------------
Q: Name a primary color.
A: You can name several primary colors depending on the col

In [5]:
# প্রশ্নের তালিকা
my_questions = [
    "What is the capital of Bangladesh?",
    "Explain Python in one sentence.",
    "Who wrote Hamlet?",
    "What is 2 + 2?",
    "Name a primary color."
]

# ব্যাচ ফাংশন কল
import time
start_time = time.time()

answers = batch_generate(my_questions)

end_time = time.time()

# ফলাফল প্রিন্ট
print("-" * 50)
for q, a in zip(my_questions, answers):
    print(f"Q: {q}")
    print(f"A: {a}")
    print("-" * 30)

print(f"Total Time Taken: {end_time - start_time:.2f} seconds")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing 5 prompts in a single batch...
--------------------------------------------------
Q: What is the capital of Bangladesh?
A: I'd be happy to tell you that the capital of Bangladesh is Dhaka.
------------------------------
Q: Explain Python in one sentence.
A: )");

Python is a high-level, interpreted programming language known for its readability and versatility, widely used for web development, data analysis, artificial intelligence, and more.
------------------------------
Q: Who wrote Hamlet?
A: To answer your question, the play "Hamlet" was written by William Shakespeare. He is one of the most famous and influential playwrights in the English language. Shakespeare wrote "Hamlet" around 1600-1601, and it is one of his most renowned tragedies.
------------------------------
Q: What is 2 + 2?
A: 2 + 2 equals 4.
------------------------------
Q: Name a primary color.
A: You can name several primary colors depending on the color theory you're referring to. In subtractive color 